## EXPLORATION_RS5
## 12. 뉴스 요약봇 만들기 [프로젝트]
### 평가문항 [상세기준]
1. Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.	[분석단계, 정제단계, 정규화와 불용어 제거, 데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.]
2. 텍스트 요약모델이 성공적으로 학습되었음을 확인하였다. [모델 학습이 진행되면서 train loss와 validation loss가 감소하는 경향을 그래프를 통해 확인했으며, 실제 요약문에 있는 핵심 단어들이 요약 문장 안에 포함되었다.]
3. Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다. [두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교하고 분석 결과를 표로 정리하여 제시하였다.]

### 12-1. 프로젝트: 뉴스기사 요약해보기

* 주요 라이브러리 버전을 확인

In [ ]:
from importlib.metadata import version
import nltk
import tensorflow
import summa
import pandas
import pandas as pd

print(nltk.__version__)
print(tensorflow.__version__)
print(pandas.__version__)
print(version('summa'))

### Step 1. 데이터 수집하기
* 뉴스 기사 데이터(news_summary_more.csv) 사용
###### sunnysai12345/News_Summary

* 데이터 다운로드

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [ ]:
data.sample(10)

### 데이터 전처리하기 (1) 데이터 정리하기

* 데이터는 기사의 본문에 해당되는 text와 headlines 두 가지 열로 구성
* 추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습
* 추출적 요약을 하는 경우에는 오직 text열만을 사용

### Step 2. 데이터 전처리하기 (추상적 요약)
* 실습에서 사용된 전처리를 참고하여 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해
* 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해볼 것

* 데이터의 중복 샘플 유무를 확인

In [ ]:
print('text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique())
print('headlines 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique())

* 중복 제외 text에는 개, headlines에는 개의 유니크한 데이터가 존재

* 데이터프레임의 drop_duplicates()를 사용하여 중복 샘플을 제거

In [ ]:
# inplace=True 를 설정하면 DataFrame 타입 값을 return 하지 않고 data 내부를 직접적으로 바꿉니다
data.drop_duplicates(subset = ['text'], inplace=True)
print('전체 샘플수 :', (len(data)))

* 중복이 제거되면서 샘플 수가 개로 감소
* 데이터 Null 값을 가지는 샘플이 있었다면, drop_duplicates()가 중복된 Null들을 지워줘도 Null 값 한 개가 남아있을 수 있음
###### 데이터프레임에 Null 값이 있는지 확인

In [ ]:
print(data.isnull().sum())

* headlines에 개의 Null 값
* 데이터프레임에서 Null을 제거할 때는 dropna() 함수를 사용

In [ ]:
data.dropna(axis=0, inplace=True)
print('전체 샘플수 :', (len(data)))

* 전체 샘플 수가 개 줄어들어 개의 샘플이 남음 
* 중복 샘플과 Null 값이 있는 샘플 제거로 만 개의 샘플 중 만 개 이상의 샘플 제거

#### 텍스트 정규화와 불용어 제거
* 텍스트 정규화(text normalization): 같은 의미인데도 다른 표현으로 쓰여 마치 다른 단어들처럼 간주되는 경우를 학습 전에 같은 표현으로 통일시켜 연산량을 줄이는 방법
* 불용어 (stopwords): 일반적으로 텍스트에는 자주 등장하지만 자연어 처리를 할 때 실질적으로 별 도움이 되지 않는 단어

* 텍스트 정규화를 위한 사전(dictionary)을 구성

In [ ]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

print("정규화 사전의 수: ", len(contractions))

* NLTK에서 제공하는 불용어 리스트를 참조해, 샘플에서 불용어를 제거

In [ ]:
print('불용어 개수 :', len(stopwords.words('english') ))
print(stopwords.words('english'))

* NLTK에서 미리 정의하여 제공하고 있는 불용어는 총 179개
* 불용어 제거 
###### NLTK를 이용해 불용어를 제거하는 파트: text 전처리 시에서만 호출하고 이미 상대적으로 문장 길이가 짧은 headlines 전처리할 때는 호출하지 않음, Abstractive한 문장 요약 결과문이 자연스러운 문장이 되려면 불용어들이 headlines에는 남아 있는 게 더 좋음
###### 해당 처리를 위해서 함수의 인자로 remove_stopwords를 추가하고, if문을 추가
* 모든 영어 문자는 소문자로
* 섞여있는 html 태그를 제거
* 정규 표현식을 통해 각종 특수문자를 제거
###### 필요한 내용만 잘 학습할 수 있도록 처리

In [ ]:
# 데이터 전처리 함수
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거 Ex) my husband (and myself!) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","", sentence) # 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah
    
    # 불용어 제거 (text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stopwords.words('english') if len(word) > 1)
    # 불용어 미제거 (headlines)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

In [ ]:
# 전처리 전, 후의 결과를 확인하기 위해서 임의의 text와 summary를 만들어 함수를 호출
temp_text = 'Everything I bought was great, infact I ordered twice and the third ordered was<br />for my mother and father.'
temp_headlines = 'Great way to start (or finish) the day!!!'

print("text: ", preprocess_sentence(temp_text))
print("headlines:", preprocess_sentence(temp_headlines, False))  # 불용어를 제거하지 않음

* 모든 알파벳이 소문자로 변환되었음 
* <br />과 같은 html 태그가 제거되었음
* (or finish)와 같은 괄호로 묶였던 단어 시퀀스가 제거되었음
* 특수문자가 제거되면서 영어만 남았음

* 훈련 데이터 전체에 대해서 전처리를 수행
###### text의 경우에는 불용어를 제거하고, headlines의 경우에는 불용어를 제거하지 않을 것이므로 따로 호출해서 진행
###### 먼저 text를 전처리하고, 결과를 확인하기 위해서 상위 5개의 줄을 출력

In [ ]:
# 전체 text 데이터에 대한 전처리 
clean_text = []

for s in data['text']:
    clean_text.append(preprocess_sentence(s))

# 전처리 후 출력
print("text 전처리 후 결과: ", clean_text[:5])

* 훈련 데이터 전체의 headlines 컬럼의 데이터를 전처리
###### headlines에 대해서 전처리 함수를 호출 시, 불용어 제거를 수행하지 않는다는 의미에서 두 번째 인자로 False

In [ ]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in data['headlines']:
    clean_headlines.append(preprocess_sentence(s, False))

print("headlines 전처리 후 결과: ", clean_headlines[:5])

* 텍스트 정제의 과정을 거친 후 다시 한번 빈(empty) 샘플이 생겼는지 확인
###### 정제 전에는 데이터가 존재했지만, 정제 과정에서 문장의 모든 단어가 사라지는 경우 샘플 자체가 빈 값을 가지게 됨
* 쉽게 확인하기 위해 데이터들을 데이터프레임에 재저장
* 빈(empty) 값을 가진 샘플들이 있다면, 모두 Null 값을 가진 샘플로 대체

In [ ]:
data['text'] = clean_text
data['headlines'] = clean_headlines

# 빈 값을 Null 값으로 변환
data.replace('', np.nan, inplace=True)

* .isnull().sum()을 사용해서 Null 값이 생겼는지 확인

In [ ]:
data.isnull().sum()

* headlines 열에서 개의 Null 값이 생겼음
* 새로 생긴 Null 샘플 모두 제거

In [ ]:
data.dropna(axis=0, inplace=True)
print('전체 샘플수 :', (len(data)))

* headlines에 개의 Null 값
* 데이터프레임에서 Null을 제거할 때는 dropna() 함수를 사용

### 데이터 전처리하기 (2) 훈련데이터와 테스트데이터 나누기
* 학습을 진행하기 위해 학습에 사용할 데이터의 크기를 결정하고, 문장의 시작과 끝을 표시
#### 샘플의 최대 길이 정하기
* 필요 없는 단어를 모두 솎아낸 데이터를 바탕으로 훈련에 사용할 샘플의 최대 길이를 정해주기
* text와 summary의 최소, 최대, 평균 길이를 구하고 길이 분포를 시각화해서 보기

In [ ]:
# 길이 분포 출력
import matplotlib.pyplot as plt

text_len = [len(s.split()) for s in data['text']]
headlines_len = [len(s.split()) for s in data['headlines']]

print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
print('요약의 최소 길이 : {}'.format(np.min(summary_len)))
print('요약의 최대 길이 : {}'.format(np.max(summary_len)))
print('요약의 평균 길이 : {}'.format(np.mean(summary_len)))

plt.subplot(1,2,1)
plt.boxplot(text_len)
plt.title('Text')
plt.subplot(1,2,2)
plt.boxplot(headlines_len)
plt.title('headlines')
plt.tight_layout()
plt.show()

plt.title('text')
plt.hist(text_len, bins = 40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

plt.title('headlines')
plt.hist(headlines_len, bins = 40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

* text의 경우 최소 길이가 , 최대 길이가  , 평균 길이가 , 그래프로 봤을 때 대체적으로는  내외의 길이를 가진다는 것을 확인
* headlines의 경우 최소 길이가 , 최대 길이가 , 평균 길이가  , 그래프로 봤을 때 대체적으로 이하의 길이를 가진다는 것을 확인
* text의 최대 길이와 headlines의 적절한 최대 길이를 임의로 정하기

In [ ]:
text_max_len = 50
headlines_max_len = 8

* 해당 길이를 선택했을 때, 얼마나 많은 샘플들을 자르지 않고 포함할 수 있는지 통계로 확인하는 편이 객관적으로 길이를 결정하는 데 도움이 될 것
* 훈련 데이터와 샘플의 길이를 입력하면, 데이터의 몇 %가 해당하는지 계산하는 함수를 만들어서 좀 더 정확하게 판단

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s.split()) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))))

* 함수를 text와 headlines에 적용해 결정한 임의의 길이가 몇%의 샘플까지 포함하는지 확인

In [ ]:
below_threshold_len(text_max_len, data['text'])
below_threshold_len(headlines_max_len,  data['headlines'])

### Step 3. 어텐션 메커니즘 사용하기 (추상적 요약)
* 일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있음 
* 어텐션 메커니즘을 사용한 seq2seq를 설계


* 50과 8로 패딩을 하게 되면 해당 길이보다 긴 샘플들은 내용이 잘리게 되는데, text 열의 경우에는 약 23%의 샘플들이 내용이 망가지게 됨
* 정해진 길이에 맞춰 자르는 것이 아니라, 정해진 길이보다 길면 제외하는 방법으로 데이터를 정제
###### text와 headlines를 담고 있는 data 데이터프레임을 위에서 임의로 정의한 text_max_len과 headlines_max_len의 길이보다 큰 샘플을 제외하는 코드

In [ ]:
data = data[data['text'].apply(lambda x: len(x.split()) <= text_max_len)]
data = data[data['headlines'].apply(lambda x: len(x.split()) <= headlines_max_len)]#

print('전체 샘플수 :', (len(data)))

#### 시작 토큰과 종료 토큰 추가하기
* seq2seq 훈련을 위해서는 디코더의 입력과 레이블에 시작 토큰과 종료 토큰을 추가해야 함
* 시작 토큰은 sostoken, 종료 토큰은 eostoken이라 임의로 명명하고 앞, 뒤로 추가
* 디코더의 입력에 해당하면서 시작 토큰이 맨 앞에 있는 문장의 이름을 decoder_input, 디코더의 출력 또는 레이블에 해당되면서 종료 토큰이 맨 뒤에 붙는 문장의 이름을 decoder_target이라고 지정 
* 두 개의 문장 모두 headlines 열로부터 생성

In [ ]:
# 요약 데이터에는 시작 토큰과 종료 토큰을 추가
data['decoder_input'] = data['headlines'].apply(lambda x : 'sostoken '+ x)
data['decoder_target'] = data['headlines'].apply(lambda x : x + ' eostoken')
data.head()

* 인코더의 입력, 디코더의 입력과 레이블을 각각 다시 Numpy 타입으로 저장

In [ ]:
encoder_input = np.array(data['text']) # 인코더의 입력
decoder_input = np.array(data['decoder_input']) # 디코더의 입력
decoder_target = np.array(data['decoder_target']) # 디코더의 레이블

*  훈련 데이터와 테스트 데이터를 분리: 분리 패키지를 사용하는 방법, 또는 직접 코딩을 통해서 분리하는 방법 등 
* encoder_input과 크기와 형태가 같은 순서가 섞인 정수 시퀀스를 생성

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

* 정수 시퀀스를 이용해 다시 데이터의 샘플 순서를 정의해 잘 섞인 샘플 생성

In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

* 섞인 데이터를 8:2의 비율로 훈련 데이터와 테스트 데이터로 분리
* 전체 데이터의 크기에서 0.2를 곱해서 테스트 데이터의 크기를 정의

In [ ]:
n_of_val = int(len(encoder_input)*0.2)
print('테스트 데이터의 수 :', n_of_val)

* 정의한 테스트 데이터의 개수를 이용해 전체 데이터를 양분
###### :표시의 위치에 주의

In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(decoder_input_train))
print('테스트 데이터의 개수 :', len(encoder_input_test))
print('테스트 레이블의 개수 :', len(decoder_input_test))

### 데이터 전처리하기 (3) 정수 인코딩
#### 단어 집합(vocabulary) 만들기 및 정수 인코딩
* 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터의 단어들을 모두 정수로 변경
###### 단어 집합(vocabulary) 생성: 각 단어에 고유한 정수를 맵핑하는 작업
* 훈련 데이터에 대해서 단어 집합을 생성
###### 원문에 해당되는 encoder_input_train에 대해서 단어 집합을 생성
###### Keras의 토크나이저를 사용하면, 입력된 훈련 데이터로부터 단어 집합을 만들 수 있음

In [ ]:
src_tokenizer = Tokenizer() # 토크나이저 정의
src_tokenizer.fit_on_texts(encoder_input_train) # 입력된 데이터로부터 단어 집합 생성

* 단어 집합이 생성되는 동시에 각 단어에 고유한 정수가 부여되었음
* 생성된 단어 집합은 src_tokenizer.word_index에 저장되어 있음
* 단어 집합에 있는 모든 단어를 사용하는 것이 아니라, 빈도수가 낮은 단어들은 훈련 데이터에서 제외하고 진행
* 등장 빈도수가 7회 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인
###### src_tokenizer.word_counts.items(): 단어와 각 단어의 등장 빈도수가 저장되어 있음, 이를 통해서 통계적인 정보를 얻을 수 있음

In [ ]:
threshold = 7
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

* encoder_input_train에는 여 개의 단어가 있음
* 등장 빈도가 threshold 값인 7회 미만, 즉 6회 이하인 단어: 단어 집합의 % 이상
* 실제로 훈련 데이터에서 등장 빈도로 차지하는 비중: %
* 등장 빈도가 6회 이하인 단어들은 정수 인코딩 과정에서 빼고, 훈련 데이터에서 제거
###### 이를 제외한 단어 집합의 크기: 여 개
###### 이와 비슷한 값으로 어림잡아 단어 집합의 크기를 으로 제한
###### 토크나이저를 정의할 때 num_words의 값을 정해, 단어 집합의 크기를 제한할 수 있음

In [ ]:
src_vocab = 
src_tokenizer = Tokenizer(num_words=src_vocab) # 단어 집합의 크기를 으로 제한
src_tokenizer.fit_on_texts(encoder_input_train) # 단어 집합 재생성

* texts_to_sequences()는 생성된 단어 집합에 기반하여 입력으로 주어진 텍스트 데이터의 단어들을 모두 정수로 변환하는 정수 인코딩을 수행
* 단어 집합의 크기를 으로 제한했으니 이 넘는 숫자들은 정수 인코딩 후에는 데이터에 존재하지 않음

In [ ]:
# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train) 
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

# 잘 진행되었는지 샘플 출력
print(encoder_input_train[:3])

* 텍스트 데이터가 아니라 정수가 나옴
* headlines 데이터에 대해서도 동일한 작업을 수행
* 케라스의 토크나이저를 사용하여 decoder_input_train을 입력으로 전체 단어 집합과 각 단어에 대한 빈도수를 계산

In [ ]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)

* 단어 집합이 생성되는 동시에 각 단어에 고유한 정수가 부여되었음
###### tar_tokenizer.word_index에 저장되어 있음
###### tar_tokenizer.word_counts.items()에 단어와 각 단어의 등장 빈도수가 저장돼 있음
* 통계적인 정보를 바탕으로, 등장 빈도수가 6회 미만인 단어들이 데이터에서 얼만큼의 비중을 차지하는지 확인

In [ ]:
threshold = 6
total_cnt = len(tar_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

* 등장 빈도가 5회 이하인 단어들은 단어 집합에서 약 %를 차지
* 훈련 데이터에서 등장 빈도로 차지하는 비중은 %
* 5회 이하 단어들은 모두 제거, 어림잡아 을 단어 집합의 크기로 제한

In [ ]:
tar_vocab = 
tar_tokenizer = Tokenizer(num_words=tar_vocab) 
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

# 텍스트 시퀀스를 정수 시퀀스로 변환
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train) 
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_target_test)

# 잘 변환되었는지 확인
print('input')
print('input ',decoder_input_train[:5])
print('target')
print('decoder ',decoder_target_train[:5])

* 정수 인코딩 작업 완료 
###### decoder_input_train과 decoder_target_train에는 숫자 이 넘는 숫자들은 존재하지 않음 

* 패딩 하기로 넘어가기 전에 점검해야 할 사항: 길이가 0이 된 샘플 제거
###### 전체 데이터에서 빈도수가 낮은 단어가 삭제됨 -> 빈도수가 낮은 단어만으로 구성되었던 샘플들은 빈(empty) 샘플이 되었을 가능성이 있음
###### 길이가 상대적으로 길었던 원문(text)의 경우에는 문제가 별로 없지만, 평균 길이가 였던 요약문(headlines)의 경우 이 현상이 크게 나타날 가능성이 높음
* 요약문에서 길이가 0이 된 샘플들의 인덱스를 수령 
###### 요약문인 decoder_input에는 sostoken 또는 decoder_target에는 eostoken이 추가된 상태: 두 토큰은 모든 샘플에서 등장하므로 빈도수가 샘플 수와 동일하게 매우 높으므로 단어 집합 제한에도 삭제되지 않음 
###### 길이가 0이 된 요약문의 실제 길이는 1로 나올 것: 길이 0이 된 decoder_input에는 sostoken, decoder_target에는 eostoken이 남아 있음
* 훈련 데이터와 테스트 데이터에 대해 요약문의 길이가 1인 경우의 인덱스를 각각 drop_train과 drop_test에 라는 변수에 저장
###### 해당 샘플 모두 삭제

In [ ]:
drop_train = [index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1]
drop_test = [index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1]

print('삭제할 훈련 데이터의 개수 :', len(drop_train))
print('삭제할 테스트 데이터의 개수 :', len(drop_test))

encoder_input_train = [sentence for index, sentence in enumerate(encoder_input_train) if index not in drop_train]
decoder_input_train = [sentence for index, sentence in enumerate(decoder_input_train) if index not in drop_train]
decoder_target_train = [sentence for index, sentence in enumerate(decoder_target_train) if index not in drop_train]

encoder_input_test = [sentence for index, sentence in enumerate(encoder_input_test) if index not in drop_test]
decoder_input_test = [sentence for index, sentence in enumerate(decoder_input_test) if index not in drop_test]
decoder_target_test = [sentence for index, sentence in enumerate(decoder_target_test) if index not in drop_test]

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(decoder_input_train))
print('테스트 데이터의 개수 :', len(encoder_input_test))
print('테스트 레이블의 개수 :', len(decoder_input_test))

* 훈련 데이터와 테스트 데이터 모두 일정량의 샘플들이 제거된 것을 확인

#### 패딩하기
* 텍스트 시퀀스를 정수 시퀀스로 변환한 후 서로 다른 길이의 샘플들을 병렬 처리하기 위해 같은 길이로 맞춰주는 패딩 작업을 해주어야 함
* 정해두었던 최대 길이로 패딩
###### 최대 길이보다 짧은 데이터들은 뒤의 공간에 숫자 0을 넣어 최대 길이로 맞춰줌

In [ ]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen=text_max_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen=text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen=summary_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen=summary_max_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen=summary_max_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen=summary_max_len, padding='post')

### 모델 설계하기
* 함수형 API를 이용해서 인코더를 설계
* 임베딩 벡터의 차원: 128, hidden state의 크기: 256
* hidden state: LSTM에서 얼만큼의 수용력(capacity)를 가질지를 정하는 파라미터 
###### LSTM의 용량의 크기나, LSTM에서의 뉴런의 개수를 나타내는 파라미터
###### 다른 신경망과 마찬가지로, 용량을 많이 준다고 해서 성능이 반드시 올라가는 것은 아님

* 인코더의 LSTM은 총 3개의 층으로 구성해서 모델의 복잡도를 높임 
###### hidden state의 크기를 늘리는 것 -> LSTM 층 1개의 용량을 늘림, 3개의 층을 사용하는 것 -> 모델의 용량을 늘림 
###### 3개의 층을 지나서 인코더로부터 나온 출력 벡터는 디코더로

* LSTM은 dropout 뿐 아니라 recurrent dropout까지 사용 가능
###### dropout: 레이어의 weight를 랜덤으로 생략하여 모델의 과적합(overfitting)을 해결해주는 방법
###### recurrent dropout: dropout을 레이어가 아닌 time step마다 해주는 방식 = time step의 입력을 랜덤으로 생략, 일반적인 dropout와 같이 regularization을 해주고, 과적합을 방지할 수 있음

* recurrent dropout을 사용할 시 cuDNN을 사용할 수 없어서 recurrent dropout을 사용하지 않을 때보다 학습 시간이 오래 걸림

![image.png](attachment:image.png)
###### dropout (왼쪽), dropout+recurrent dropout (=variational dropout, 오른쪽)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# 인코더 설계 시작
embedding_dim = 128
hidden_size = 256

# 인코더
encoder_inputs = Input(shape=(text_max_len,))

# 인코더의 임베딩 층
enc_emb = Embedding(src_vocab, embedding_dim)(encoder_inputs)

# 인코더의 LSTM 1
# encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True ,dropout = 0.4, recurrent_dropout = 0.4)
encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True ,dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
# encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
# encoder_lstm3 = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)
encoder_lstm3 = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

* 디코더를 설계
* 디코더의 임베딩 층과 LSTM을 설계하는 것은 인코더와 거의 동일
* LSTM의 입력을 정의할 때, initial_state의 인자값으로 인코더의 hidden state와 cell state의 값을 넣어줘야 함

In [ ]:
# 디코더 설계
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb_layer = Embedding(tar_vocab, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

# 디코더의 LSTM
# decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

* 디코더의 출력층을 설계

In [ ]:
# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs) 

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

* 디코더의 출력층에서는 headlines의 단어장인 tar_vocab의 수많은 선택지 중 하나의 단어를 선택하는 다중 클래스 분류 문제를 풀어야 함
###### Dense의 인자로 tar_vocab을 주고, 활성화 함수로 소프트맥스 함수를 사용

* 어텐션 메커니즘: 인코더의 hidden state와 cell state를 디코더의 초기 state로 사용하는 가장 기본적인 seq2seq에서 디코더의 출력층을 설계를 살짝 바꿔서 성능을 높일 수 있는 방법

#### 어텐션 메커니즘
* 어텐션 메커니즘을 수행하는 어텐션 함수를 설계하는 것 = 다른 새로운 신경망을 설계해야 함 

* TensorFlow에 이미 구현된 어텐션 함수를 가져와서 디코더의 출력층에 결합
###### Bahdanau 스타일의 어텐션 사용 

* 어텐션 층을 만들고, 앞서 설계한 디코더의 출력층을 수정
###### 인코더의 hidden state들과 디코더의 hidden state들을 어텐션 함수의 입력으로 사용하고, 어텐션 함수가 리턴한 값을 예측 시에 디코더의 hidden state와 함께 활용하는 형태로 작동

In [ ]:
from tensorflow.keras.layers import AdditiveAttention

# 어텐션 층(어텐션 함수)
attn_layer = AdditiveAttention(name='attention_layer')

# 인코더와 디코더의 모든 time step의 hidden state를 어텐션 층에 전달하고 결과를 리턴
attn_out = attn_layer([decoder_outputs, encoder_outputs])


# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

### 모델 훈련하기
* 설계한 모델을 가지고 훈련을 진행
* EarlyStopping: '조기 종료', 특정 조건이 충족되면 훈련을 멈추는 역할
###### es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
* val_loss(검증 데이터의 손실)을 관찰하다가, 검증 데이터의 손실이 줄어들지 않고 증가하는 현상이 2회(patience=2) 관측되면 학습을 멈추도록 설정되어 있음 
* EarlyStopping이 작동한다면 epochs가 아무리 크게 설정되어 있어도 모델 훈련을 최적점에서 멈출 수 있음
* Early Stopping을 사용할 경우 조심해야 하는 경우:
###### patience가 0이 아닌 경우 훈련이 종료되었을 때 성능이 최고인 상황이 아닐 수 있음
###### 학습 중에 모델을 저장하는 callback 함수를 같이 사용하여 방지

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), \
          batch_size=256, callbacks=[es], epochs=50)

* 훈련 데이터의 손실과 검증 데이터의 손실이 줄어드는 과정을 시각화

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

### 인퍼런스 모델 구현하기
* 테스트 단계에서는 정수 인덱스 행렬로 존재하던 텍스트 데이터를 실제 데이터로 복원해야 하므로, 필요한 3개의 사전을 미리 준비

In [ ]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

* seq2seq는 훈련할 때와 실제 동작할 때(인퍼런스 단계)의 방식이 다르므로 그에 맞게 모델 설계를 별개로 진행해야 함
###### 훈련 단계에서는 디코더의 입력부에 정답이 되는 문장 전체를 한꺼번에 넣고 디코더의 출력과 한 번에 비교할 수 있으므로, 인코더와 디코더를 엮은 통짜 모델 하나만 준비
###### 정답 문장이 없는 *인퍼런스 단계에서는 만들어야 할 문장의 길이만큼 디코더가 반복 구조로 동작해야 하기 때문에 부득이하게 인퍼런스를 위한 모델 설계를 별도로 해주어야 함*
###### 이때는 인코더 모델과 디코더 모델을 분리해서 설계

In [ ]:
# 인코더 설계
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2 = dec_emb_layer(decoder_inputs)

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

* 어텐션 메커니즘을 사용하는 출력층을 설계

In [ ]:
# 어텐션 함수
decoder_hidden_state_input = Input(shape=(text_max_len, hidden_size))
attn_out_inf = attn_layer([decoder_outputs2, decoder_hidden_state_input])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

print('=3')

* 인퍼런스 단계에서 단어 시퀀스를 완성하는 함수 생성

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_word_to_index['sostoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_index_to_word[sampled_token_index]

        if (sampled_token!='eostoken'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == 'eostoken'  or len(decoded_sentence.split()) >= (headlines_max_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence

### 모델 테스트하기
* 테스트 단계에서는 정수 시퀀스를 텍스트 시퀀스로 변환하여 결과를 확인하는 것이 편함 -> 주어진 정수 시퀀스를 텍스트 시퀀스로 변환하는 함수를 생성 
###### 함수를 만들 때, text의 정수 시퀀스에서는 패딩을 위해 사용되는 숫자 0을 제외하고 headlines의 정수 시퀀스에서는 숫자 0, 시작 토큰의 인덱스, 종료 토큰의 인덱스를 출력에서 제외하도록 만들 것

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if (i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2headlines(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_word_to_index['sostoken']) and i!=tar_word_to_index['eostoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp 

* 테스트 데이터 약 50개의 샘플에 대해서 실제 요약과 예측된 요약을 비교

In [ ]:
for i in range(50, 100):
    print("원문 :", seq2text(encoder_input_test[i]))
    print("실제 요약 :", seq2headlines(decoder_input_test[i]))
    print("예측 요약 :", decode_sequence(encoder_input_test[i].reshape(1, text_max_len)))
    print("\n")

* 기존의 요약과는 다른 요약을 출력하면서도 원문의 내용을 담고 있는 의미 있는 요약들이 보임 
###### 일부 요약의 경우에는 원문에 없던 단어를 사용해서 요약을 하기도 함 -> 워드 임베딩과 RNN의 혼합 효과

* 성능을 개선 방안:
###### seq2seq와 어텐션의 자체의 조합을 좀 더 좋게 수정하는 방법
###### 빔 서치(beam search)
###### 사전 훈련된 워드 임베딩(pre-trained word embedding)
###### 인코더 - 디코더 자체의 구조를 새로이 변경한 하는 트랜스포머(Transformer)

### Step 4. 실제 결과와 요약문 비교하기 (추상적 요약)
* 원래의 요약문(headlines 열)과 학습을 통해 얻은 추상적 요약의 결과를 비교



### Step 5. Summa을 이용해서 추출적 요약해보기
* 추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높음 
* 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮음
* Summa의 summarize를 사용하여 추출적 요약 실행

In [ ]:
import requests
from summa.summarizer import summarize

In [ ]:
# 매트릭스 시놉시스를 다운로드
text = requests.get('http://rare-technologies.com/the_matrix_synopsis.txt').text

* text에는 매트릭스 시놉시스가 문자열로 저장되어 있음 
* 출력 결과가 아주 길기 때문에 일부만 출력해보고, 잘 저장이 되었는지 확인

In [ ]:
print(text[:1500])

#### summarize 사용하기
* Summa의 summarize()의 인자로 사용되는 값:
###### text (str) : 요약할 테스트
###### ratio (float, optional) – 요약문에서 원본에서 선택되는 문장 비율 (0~1 사이값)
###### words (int or None, optional) (출력에 포함할 단어 수)
###### ratio와 함께 두 파라미터가 모두 제공되는 경우 ratio는 무시
###### split (bool, optional) (True면 문장 list / False는 조인(join)된 문자열을 반환)

* Summa의 summarize는 문장 토큰화를 별도로 하지 않더라도 내부적으로 문장 토큰화를 수행
###### 문장 구분이 되어있지 않은 원문을 바로 입력으로 넣을 수 있음 
###### 비율을 적게 주어서 요약문으로 선택되는 문장의 개수를 줄임
###### 원문의 0.005%만을 출력하도록 설정

In [ ]:
print('headlines:')
print(headlines(text, ratio=0.005))

* 리스트로 출력 결과를 받고 싶다면 split 인자의 값을 True로 설정

In [ ]:
print('headlines:')
print(headlines(text, ratio=0.005, split=True))

* 단어의 수로 요약문의 크기를 조절할 수 있음 
* 단어를 50개만 선택

In [ ]:
print('headlines:')
print(headlines(text, words=50))